In [ ]:
import os
import pathlib
import optuna

SAVE = False
WIDTH = 1200
SCALE = 3
ID = 13
study = optuna.load_study(study_name=f'experiment_{ID}', storage='sqlite:///{}'.format(os.path.join('..', 'logs', 'experiment', 'studies.db')))
if SAVE:
    path = pathlib.Path(os.path.join('images', 'optuna_study_exploration'))
    path.mkdir(parents=True, exist_ok=True)

In [ ]:
# If the study is stopped by e.g. system timeout on kaggle or colab the last trial can get stuck in the running state and become "zombie trial".
# Setting fix_running_trials to True will set every running trial to failed one. This may be necessary if study is to be re-run to compute more trials.
# https://github.com/optuna/optuna/issues/685
# https://github.com/optuna/optuna/pull/1214
fix_running_trials = False
running_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.RUNNING])
print(running_trials)
if fix_running_trials:
    for trial in running_trials:
        study._storage.set_trial_state(trial_id=trial._trial_id, state=optuna.trial.TrialState.FAIL)

In [ ]:
len(study.get_trials(deepcopy=False))

In [ ]:
study.best_trial

In [ ]:
translate_dict = {
    'SGD_momentum': 'SGD momentum',
    'SGD_nesterov': 'SGD nesterov',
    'SGD_weight_decay': 'SGD rozkład wag',
    'learning_rate': 'tempo uczenia',
    'ReduceLROnPlateau...': 'współczynnik lr',
    'AP': 'AP IoU=.50:.05:.95',
    'ReduceLROnPlateau_factor': 'współczynnik lr',
    'beta1': 'beta 1',
    'beta2': 'beta 2',
    'epsilon': 'epsilon',
    'blur': 'blur',
    'h_flip': 'horyzontalne odbicie',
    'rotate': 'rotacja',
    'random_crop': 'losowe wycięcie',
    'gauss_noise': 'szum Gaussa',
    'equalize': 'wyrównanie histogramu',
}

In [ ]:
fig = optuna.visualization.plot_intermediate_values(study)
fig.update_layout(title_text='', xaxis_title_text='Epoka', yaxis_title_text='AP IoU=.50:.05:.95',  margin=dict(l=70, r=50, t=30, b=80))
fig.show()
if SAVE:
    fig.write_image(os.path.join('images', 'optuna_study_exploration', f'intermediate_values_{ID}.jpg'), width=WIDTH, scale=SCALE)

In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(title_text='', xaxis_title_text='Próby', yaxis_title_text='AP IoU=.50:.05:.95', margin=dict(l=70, r=70, t=30, b=80))
fig.data[0].name='AP danej próby'
fig.data[1].name='Najlepsze AP'
fig.show()
if SAVE:
    fig.write_image(os.path.join('images', 'optuna_study_exploration', f'optimization_history_{ID}.jpg'), width=WIDTH, scale=SCALE)

In [ ]:
fig = optuna.visualization.plot_parallel_coordinate(study, target_name='AP')
for dim in fig.data[0].dimensions:
    if dim.label in ['SGD_nesterov', 'blur', 'h_flip', 'rotate', 'random_crop', 'gauss_noise', 'equalize']:
        new_ticktext = []
        for tick in dim.ticktext:
            if tick == True:
                new_ticktext.append('Tak')
            elif tick == False:
                new_ticktext.append('Nie')
        dim.ticktext = new_ticktext
for dim in fig.data[0].dimensions:
    dim.label = translate_dict[dim.label]
fig.update_layout(title_text='', margin=dict(l=70, r=70, t=30, b=90))
fig.show()
if SAVE:
    fig.write_image(os.path.join('images', 'optuna_study_exploration', f'parallel_coordinate_{ID}.jpg'), width=WIDTH, scale=SCALE)

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.data[0].y = tuple(translate_dict[label] for label in fig.data[0].y)
fig.update_layout(title_text='', xaxis_title_text='Wpływ na wartość AP IoU=.50:.05:.95', yaxis_title_text='Hiperparametry', margin=dict(l=30, r=30, t=30, b=30))
fig.show()
if SAVE:
    fig.write_image(os.path.join('images', 'optuna_study_exploration', f'param_importancess_{ID}.jpg'), width=WIDTH, scale=SCALE)